In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload


## Graph


In [ ]:
from graphviz import Digraph


class Graph:
    def draw(self, root):
        def trace(_value):
            # Set of all nodes and edges in a graph.
            nodes, edges = set(), set()

            def build(v):
                if v not in nodes:
                    nodes.add(v)
                    for child in v.children:
                        if child is not None:
                            edges.add((child, v))
                            build(child)
            build(_value)
            return nodes, edges

        graph = Digraph(format='svg', graph_attr={'rankdir': 'LR'})

        node, edges = trace(root)

        for value in node:
            if (value.data is None):
                continue

            uid = str(id(value))
            # Create a rectangular node for each value.
            graph.node(name=uid, label="{ " + "{label} | data {data:.4f} | grad {grad:.4f}".format(
                label=value.label, data=value.data or 0, grad=value.grad) + " }", shape='record', )

            if value._op:
                # If this value is the result of an operation, create a node for the operation.
                graph.node(name=uid + value._op, label=value._op,
                           color='gray', shape='circle', fontcolor='gray')
                # and connect this node to it.
                graph.edge(uid + value._op, uid)

        for v1, v2 in edges:
            if (v1.data is None) or (v2.data is None):
                continue

            # Connect 1st value to the op node of the 2nd value.
            graph.edge(str(id(v1)), str(id(v2)) + v2._op)

        return graph

    def __call__(self, root):
        return self.draw(root)


graph = Graph()


## Value


In [ ]:
from typing import Callable, TypeVar


TValue = TypeVar("TValue", bound="Value")


class Value:
    """
    A node in the computation graph.
    """
    data: float
    """The value of this node"""
    grad: float
    """The gradient of this node"""
    children: tuple[TValue]
    """The children nodes in the computation graph"""
    _backward: Callable[..., None]
    """Go backwards in the computation graph and compute the gradient of this node."""
    _op: str
    """The operation symbol for this node"""
    label: str
    """An optional label for this node"""

    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0
        self.children = _children
        self._backward = lambda: None
        self._op = _op
        self.label = label

    def __repr__(self):
        return f'Value(data={self.data})'

    def _topological_sort(self, root: TValue):
        """Perform a topological sort, going backwards from the given node. Returns a list of nodes in the reversed order of execution."""

        def build(v: TValue):
            if v not in visited:
                visited.add(v)
                for child in v.children:
                    if isinstance(child, Value):
                        build(child)
                sorted.append(v)

        sorted: list[TValue] = []
        visited: set[TValue] = set()

        build(root)

        return reversed(sorted)

    def backPropagate(self):
        """Perform backpropagation on the computation graph"""
        sorted = self._topological_sort(self)

        self.grad = 1.0  # Start with the gradient of 1 for the root node.

        for v in sorted:
            v._backward()

    def _operate(self, other: TValue, fn: callable, op: str, backward=lambda: None):
        other = Value(other) if (not isinstance(
            other, Value) and not None) else other

        # Create a new node from applying the given operation between 'self' and 'other'.
        next = Value(fn(self.data, other.data),
                     (self, other), op)

        # https://www.mathsisfun.com/calculus/derivatives-rules.html
        next._backward = lambda: backward(next, self, other)

        return next

    def __neg__(self): return self * -1

    # Below are overloaded operators for basic mathematical operations:
    def __truediv__(self, other):
        def _backward(next, _self, _other):
            _self.grad = 1.0 / _other.data * next.grad
            _other.grad = -_self.data / _other.data**2 * next.grad

        return self._operate(other, lambda x, y: x * y ** -1, '/', _backward)

    def __add__(self, other):
        """Add two values together."""
        def _backward(next, _self, _other):
            _self.grad += 1.0 * next.grad
            _other.grad += 1.0 * next.grad

        return self._operate(other, lambda x, y: x + y, '+', _backward)

    def __radd__(self, other): return self + other

    def __sub__(self, other): return self + (-other)

    def __mul__(self, other):
        """Multiply two values."""
        def _backward(next, _self, _other):
            _self.grad += _other.data * next.grad
            _other.grad += _self.data * next.grad

        return self._operate(other, lambda x, y: x * y, '*', _backward)

    def __rmul__(self, other): return self * other

    def __pow__(self, other):
        """Raise a value to a power."""
        def _backward(next, _self, _other):
            _self.grad += _other.data * \
                _self.data ** (_other.data - 1) * next.grad

        return self._operate(other, lambda x, y: x ** y, '**', _backward)

    def exp(self):
        """Exponential of a value."""
        def _backward(next, _self, _other=None):
            _self.grad = next.data * next.grad

        return self._operate(None, lambda x, y=None: math.exp(x), 'exp', _backward)

    def tanh(self):
        """Hyperbolic tangent of a value."""
        def _backward(next, _self, _other=None):
            _self.grad = (1 - math.tanh(_self.data)**2) * next.grad

        return self._operate(None, lambda x, y=None: (math.exp(2*x) - 1) / (math.exp(2*x) + 1), 'tanh', _backward)


In [ ]:
# Neuron inputs: x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# Neuron weights: w1, w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# Bias
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1 * w1
x1w1.label = 'x1*w1'
x2w2 = x2 * w2
x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2
x1w1x2w2.label = 'x1*w1 + x2*w2'

# Neuron
n = x1w1x2w2 + b
n.label = 'n'

o = n.tanh()
o.label = 'o'

o.backPropagate()

graph(o)


In [ ]:
# Neuron inputs: x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# Neuron weights: w1, w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# Bias
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1 * w1
x1w1.label = 'x1*w1'
x2w2 = x2 * w2
x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2
x1w1x2w2.label = 'x1*w1 + x2*w2'

# Neuron
n = x1w1x2w2 + b
n.label = 'n'

e: Value = (n*2).exp()
o = (e - 1) / (e + 1)
o.label = 'o'

o.backPropagate()
graph(o)
